In [3]:
from platform import python_version

print('python version of this jupyter notebook:', python_version())

python version of this jupyter notebook: 3.7.10


In [4]:
import pandas as pd
from tqdm.notebook import tqdm
from collections import defaultdict

In [7]:
movies = pd.read_csv('./archive/IMDb movies.csv')

/Users/xuezhuoer/miniconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3444: DtypeWarning: Columns (3) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [8]:
print('the length of the dataset is:', len(movies))

the length of the dataset is: 85855


In [9]:
# use 70000 of the dataset as the training set and the rest as the testing set
train_set = movies[:70000]
test_set = movies[70000:]

In [10]:
train_set.head()

,imdb_title_id,title,original_title,year,date_published,genre,duration,country,language,director,...,actors,description,avg_vote,votes,budget,usa_gross_income,worlwide_gross_income,metascore,reviews_from_users,reviews_from_critics
0,tt0000009,Miss Jerry,Miss Jerry,1894,1894-10-09,Romance,45,USA,None,Alexander Black,...,"Blanche Bayliss, William Courtenay, Chauncey D...",The adventures of a female reporter in the 1890s.,5.9,154,NaN,NaN,NaN,NaN,1.0,2.0
1,tt0000574,The Story of the Kelly Gang,The Story of the Kelly Gang,1906,1906-12-26,"Biography, Crime, Drama",70,Australia,None,Charles Tait,...,"Elizabeth Tait, John Tait, Norman Campbell, Be...",True story of notorious Australian outlaw Ned ...,6.1,589,$ 2250,NaN,NaN,NaN,7.0,7.0
2,tt0001892,Den sorte drøm,Den sorte drøm,1911,1911-08-19,Drama,53,"Germany, Denmark",NaN,Urban Gad,...,"Asta Nielsen, Valdemar Psilander, Gunnar Helse...",Two men of high rank are both wooing the beaut...,5.8,188,NaN,NaN,NaN,NaN,5.0,2.0
3,tt0002101,Cleopatra,Cleopatra,1912,1912-11-13,"Drama, History",100,USA,English,Charles L. Gaskill,...,"Helen Gardner, Pearl Sindelar, Miss Fielding, ...",The fabled queen of Egypt's affair with Roman ...,5.2,446,$ 45000,NaN,NaN,NaN,25.0,3.0
4,tt0002130,L'Inferno,L'Inferno,1911,1911-03-06,"Adventure, Drama, Fantasy",68,Italy,Italian,"Francesco Bertolini, Adolfo Padovan",...,"Salvatore Papa, Arturo Pirovano, Giuseppe de L...",Loosely adapted from Dante's Divine Comedy and...,7.0,2237,NaN,NaN,NaN,NaN,31.0,14.0


In [11]:
actor_movies =  {}
actor_count = {}
vote_add = 0
for index, row in tqdm(train_set.iterrows()):
    if type(row['actors']) != str:
        continue
    actors = row['actors'].split(',')
    for a in actors:
        if a not in actor_movies:
            actor_movies[a] = {row['imdb_title_id']}
            actor_count[a] = {0}
        else:
            actor_movies[a].add(row['imdb_title_id'])
            actor_count[a] 
    vote_add += row['avg_vote']
    
vote_mean = vote_add/len(train_set)

0it [00:00, ?it/s]

In [12]:
# count the number of movies an actor plays
actor_count = defaultdict(int)
actor_rate = defaultdict(int)

for index, row in tqdm(train_set.iterrows()):
    if type(row['actors']) != str:
        continue
    actors = row['actors'].split(',')
    for a in actors:
        a = a.replace(" ", "")
        actor_count[a] += 1
        actor_rate[a] += row['avg_vote']
        if a == "Angibell": print(row['title'], row['avg_vote'])

0it [00:00, ?it/s]

Lejos de Casa pelicula Venezolana 9.8


In [13]:
for a in actor_rate:
    actor_rate[a] = actor_rate[a]/actor_count[a]

In [14]:
actor_rate = sorted(actor_rate.items(), key= lambda x: x[1], reverse=True)

45


In [15]:
actor_count = sorted(actor_count.items(), key= lambda x: x[1], reverse=True)

In [59]:
# global count
# count = [0] * 10
def feata(d):
#     global count
    year = [0] * 11
    movie_year = d['year']
#     print(movie_year)
    if int(movie_year) <= 1930:
        year[0] = 1
#         count[0] += 1
    else:
        one_hot = int((int(movie_year) - 1921)/10)
        year[one_hot] = 1
#         count[one_hot] += 1
        
    return year[:-1]

In [60]:
X = [feata(row) for index, row in tqdm(train_set.iterrows())]
print(count)


[1057, 2840, 3014, 4144, 5151, 6192, 7432, 10159, 18684, 11327]


In [16]:
genres = []
genreSet = set()
for index, row in train_set.iterrows():
    genres.append([])
    for t in row['genre'].split(', '):
        genreSet.add(t)
        genres[-1].append(t)
numGenre = len(genreSet)
genreToIndex = dict()
for i, g in enumerate(list(genreSet)):
    genreToIndex[g] = i
for i, genre in enumerate(genres):
    t = [0] * numGenre
    for g in genre:
        t[genreToIndex[g]] = 1
    genres[i] = t
    
print(genres)

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



In [93]:
countries = []
tmp = set()
frequency = defaultdict(int)
for index, row in train_set.iterrows():
    countries.append([])
#     print(index, row['country'], row['title'])
    if type(row['country']) != str: continue
    for t in row['country'].split(', '):
        tmp.add(t)
        frequency[t] += 1
        countries[-1].append(t)
numCountries = len(tmp)
frequency = sorted(frequency.keys(), key=lambda x: frequency[x], reverse=True)[:10]
print(frequency)
countryToIndex = dict()
for i, c in enumerate(frequency):
    countryToIndex[c] = i
for i, country in enumerate(countries):
    t = [0] * 10
    for c in country:
        if c in frequency:
            t[countryToIndex[c]] = 1
    countries[i] = t
    
# print(countries)

['USA', 'France', 'UK', 'Italy', 'India', 'Japan', 'Germany', 'Canada', 'Spain', 'Hong Kong']


In [96]:
print(countries[2000])
for index, row in train_set.iterrows():
    if index == 2000: 
        print(row['country'])
        break

[1, 0, 0, 0, 0, 0, 0, 0, 0, 0]
USA


In [33]:
directorAndHisGenre = defaultdict(list)
m = 0

for index, row in train_set.iterrows():
    if type(row['director']) != str: continue 
    for t in row['director'].split(', '):
        if len(directorAndHisGenre[t]) == 0: directorAndHisGenre[t] = [0 for _ in range(2 * numGenre)]
        for g in row['genre'].split(', '):
            directorAndHisGenre[t][genreToIndex[g]] += 0.1
            directorAndHisGenre[t][genreToIndex[g]+numGenre] += row['avg_vote'] / 20
            m = max(m, directorAndHisGenre[t][genreToIndex[g]+numGenre])
        
print(directorAndHisGenre)

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



In [34]:
directorAndHisGenre['Paul Wegener']

[0,
 0,
 0,
 0.1,
 0,
 0,
 0,
 0,
 0,
 0.30000000000000004,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0.2,
 0,
 0,
 0,
 0.325,
 0,
 0,
 0,
 0,
 0,
 1.025,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0.685]

print(m)
directorAndHisGenre['Paul Wegener']

In [27]:
genreStat = [sum([n[i] for n in genres]) / len(genres[0]) for i in range(len(genres[0]))]
print(genreStat)
genreToIndex
print(sum(genreStat))

[423.6, 126.28, 0.08, 1544.84, 481.08, 0.08, 0.04, 76.56, 34.48, 293.48, 166.88, 81.32, 383.48, 259.24, 58.6, 119.24, 75.16, 966.12, 26.52, 0.04, 63.76, 74.68, 346.6, 60.52, 126.76]
5789.440000000002


In [42]:
actorAndHisGenre = defaultdict(list)
m = 0

for index, row in train_set.iterrows():
    if type(row['actors']) != str: continue 
    for t in row['actors'].split(', '):
        tmpName = t.replace(' ', '')
        if len(actorAndHisGenre[tmpName]) == 0: actorAndHisGenre[tmpName] = [0 for _ in range(2 * numGenre)]
        for g in row['genre'].split(', '):
            actorAndHisGenre[tmpName][genreToIndex[g]] += 0.1
            actorAndHisGenre[tmpName][genreToIndex[g]+numGenre] += row['avg_vote'] / 20
#             m = max(m, directorAndHisGenre[t][genreToIndex[g]+numGenre])
        
print(actorAndHisGenre.keys())

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



In [45]:
print(list(actorAndHisGenre.keys())[:10])
actorAndHisGenre['BlancheBayliss']

['BlancheBayliss', 'WilliamCourtenay', 'ChaunceyDepew', 'ElizabethTait', 'JohnTait', 'NormanCampbell', 'BellaCola', 'WillCoyne', 'SamCrewes', 'JackEnnis']


[0,
 0,
 0,
 0,
 0.1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0.29500000000000004,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0]

In [46]:
years = []
for index, row in tqdm(train_set.iterrows()):
    t = [0] * 10
    year = row['year']
    i = 9
    if int(year) <= 1930:
        i = 1
    else: 
        i = int((int(year) - 1921)/10)
    t[i] = 1
    years.append(t)

0it [00:00, ?it/s]

In [58]:
X = []
for index, row in tqdm(train_set.iterrows()):
    X.append(years[index] + directorAndHisGenre[row['director']] + genres[index])
    actorFeature = [0 for _ in range(2 * numGenre)]
    if type(row['actors']) == str: 
        for actor in row['actors'].split(', '):
            for i, x in enumerate(actorAndHisGenre[actor.replace(' ', '')]):
                actorFeature[i] += x
    X[-1] = X[-1] + actorFeature



0it [00:00, ?it/s]